In [32]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import urllib2, json, argparse
import logging, os, io, requests
import key # import api key
from pprint import pprint
from jsonmerge import merge
import glob
import itertools
from bs4 import BeautifulSoup

# force utf8 encoding
import sys  
reload(sys)  
sys.setdefaultencoding('utf8')

# make plots show up inline
%matplotlib inline 

In [2]:
api_key = key.API_KEY
base_url = 'http://www.giantbomb.com/api'

In [3]:
offset=0
payload = {'api_key': api_key, 'format':'json', 'offset':offset, 'limit':1}

In [4]:
offsets = [0, 101, 201, 301, 401, 501, 601]
gb = []
for x in offsets:
    payload = {'api_key': api_key, 'format':'json', 'offset':x}
    results = requests.get(base_url + "/reviews/", params=payload).json()['results']
    gb.append(results)
    # because giantbomb limits us to 100 records at a time,
    # save each record as a separate file
    with io.open('%s.json' % x, 'w', encoding='utf-8') as f:
        f.write(unicode(json.dumps(results, ensure_ascii=False)))

In [33]:
result = []
for offset in offsets:
    with open(str(offset) + '.json') as data_file:
        result.append(json.load(data_file))

In [34]:
# use itertools to extract double bracketed list
merged_list = list(itertools.chain(*result))

df = pd.DataFrame(merged_list)

In [16]:
df.to_csv('giantbomb_reviews.csv', encoding='utf-8')

In [35]:
with open("merged_files.json", "wb") as outfile:
     json.dump(merged_list, outfile)

In [46]:
# strip html from description field
for index, row in df.iterrows():
    html = row['description']
    soup = BeautifulSoup(html)
    text = soup.getText()
    df['description_strip'][index] = text

In [97]:
game = df['game']
game = game.apply(pd.Series)
game.drop('api_detail_url', axis=1, inplace=True)

In [98]:
game.head(5)

,id,name,site_detail_url
0,20462,SingStar,http://www.giantbomb.com/singstar/3030-20462/
1,20455,Haze,http://www.giantbomb.com/haze/3030-20455/
2,20669,Metal Gear Solid 4: Guns of the Patriots,http://www.giantbomb.com/metal-gear-solid-4-gu...
3,20681,Poker Smash,http://www.giantbomb.com/poker-smash/3030-20681/
4,20655,Battlefield: Bad Company,http://www.giantbomb.com/battlefield-bad-compa...


In [132]:
merged = pd.concat([df, game], axis=1)
merged['site'] = 'Giant Bomb'

In [133]:
giantbomb = merged[['name', 
                      'publish_date',  
                      'reviewer',
                      'api_detail_url',
                      'description_strip',
                      'site',
                      'platforms',
                      'score' 
                      ]]

In [134]:
giantbomb.columns = ['game', 'date', 'reviewer', 
                     'link', 'review', 'site', 
                     'platforms', 'score']

In [135]:
# get names of reivewers
pd.Series(giantbomb['reviewer'].values.ravel()).unique()

array([u'Jeff', u'Brad', u'Ryan', u'Vinny', u'Andy', u'Dave', u'Drew',
       u'Alex', u'Matt', u'Patrick', u'Dan'], dtype=object)

In [136]:
# Full names for Reviewers
giantbomb.ix[giantbomb.reviewer == "Jeff", 'reviewer'] = 'Jeff Gerstmann'
giantbomb.ix[giantbomb.reviewer == "Brad", 'reviewer'] = 'Brad Shoemaker'
giantbomb.ix[giantbomb.reviewer == "Ryan", 'reviewer'] = 'Ryan Davis'
giantbomb.ix[giantbomb.reviewer == "Vinny", 'reviewer'] = 'Vinny Caravella'
giantbomb.ix[giantbomb.reviewer == "Andy", 'reviewer'] = 'Andy McCurdy'
giantbomb.ix[giantbomb.reviewer == "Dave", 'reviewer'] = 'Dave Lang'
giantbomb.ix[giantbomb.reviewer == "Drew", 'reviewer'] = 'Drew Scanlon'
giantbomb.ix[giantbomb.reviewer == "Alex", 'reviewer'] = 'Alex Navarro'
giantbomb.ix[giantbomb.reviewer == "Matt", 'reviewer'] = 'Matthew Rorie'
giantbomb.ix[giantbomb.reviewer == "Patrick", 'reviewer'] = 'Patrick Klepek'
giantbomb.ix[giantbomb.reviewer == "Dan", 'reviewer'] = 'Dan Ryckert'

In [137]:
giantbomb['score_100'] = giantbomb['score']*20

In [138]:
giantbomb['score_100']

0       80
1       40
2      100
3       80
4      100
5       60
6       60
7      100
8      100
9       80
10      60
11      40
12      80
13      80
14     100
15      40
16      60
17      60
18      40
19      80
20      40
21      40
22      40
23      60
24     100
25      60
26      40
27      80
28      60
29      80
      ... 
629     60
630     60
631     80
632     60
633    100
634     80
635     80
636     80
637     40
638    100
639     60
640     40
641     80
642     80
643     60
644     40
645     40
646     80
647     80
648     40
649     80
650     60
651     80
652     60
653     40
654     40
655     60
656     60
657    100
658     80
Name: score_100, dtype: int64

In [140]:
giantbomb.to_csv('giantbomb_reviews.csv', encoding='utf-8', index=False)